Load data from recipe runs.

In [1]:
%use dataframe
val projectIds = DataFrame.read("/Users/merlin/IdeaProjects/private/Release-Train-Metro-Plan/src/main/kotlin/data/ProjectCoordinates.csv")
val dependencies = DataFrame.read("/Users/merlin/IdeaProjects/private/Release-Train-Metro-Plan/src/main/kotlin/data/DependenciesInUse.csv")

print("Loaded ${projectIds.rowsCount()} projects and ${dependencies.rowsCount()} dependencies.")

Loaded 78 projects and 49658 dependencies.

In [5]:
import java.nio.file.Files
import java.nio.file.StandardOpenOption
import kotlin.io.path.Path
import kotlin.io.path.createFile

data class Artifact(val group: String?,
                   val artifact: String,
                   var parent: Artifact? = null) {
    override fun equals(other: Any?): Boolean = other is Artifact && other.group == group && other.artifact == artifact
    override fun hashCode(): Int = group.hashCode() * 31 + artifact.hashCode()
}

data class Repository(val path: String, val artifacts: Set<Artifact>,  val dependencies: Set<Artifact>) {
    override fun equals(other: Any?): Boolean = other is Repository && other.path == path
    override fun hashCode(): Int = path.hashCode()
}

val repos = mutableListOf<Repository>()

//val inOrgDeps = dependencies
//    .select { it["repositoryPath", "repositoryBranch", "groupId", "artifactId"] }
//    .filter { it.repositoryBranch == "master" || it.repositoryBranch == "main" }
//    .groupBy{ it["repositoryPath"] }

projectIds
    .select { it["repositoryPath", "repositoryBranch", "groupId", "artifactId"] }
    .filter { it.repositoryBranch == "master" || it.repositoryBranch == "main" }
    .groupBy { it["repositoryPath"]}
    .forEach {
        repos.add(Repository(it.key.repositoryPath,
            it.group
                .map {
                    a -> Artifact(a.groupId, a.artifactId) }
                .toSet(),
            dependencies
                .select { d-> d["repositoryPath", "repositoryBranch", "groupId", "artifactId"] }
                .filter { d -> d.repositoryBranch == "master" || d.repositoryBranch == "main" }
                .filter { d -> repositoryPath == it.key.repositoryPath  }
                .map { d -> Artifact(d.groupId, d.artifactId) }
                .toSet()
        ))
    }

//parentIds
//    .select { it["repositoryPath", "repositoryBranch", "projectArtifactId", "groupId", "artifactId"] }
//    .filter { it.repositoryBranch == "master" || it.repositoryBranch == "main" }
//    .forEach {
//        repos.firstOrNull { r -> r.path == it.repositoryPath }
//            ?.artifacts?.firstOrNull { a -> a.artifact == it.projectArtifactId }
//            ?.let { a -> a.parent = Artifact(it.groupId, it.artifactId)}
//    }

println("derived ${repos.size} repositories from the data, containing ${repos.sumOf { it.artifacts.size }} artifacts and ${repos.sumOf { it.dependencies.size }} dependencies.")

derived 38 repositories from the data, containing 71 artifacts and 719 dependencies.


In [3]:
// all data as one
/*
println(
"""
@startuml
${repos.joinToString("\n") { it.asPlantUml() }}
${repos.joinToString("\n") { it.asParentRelationships() }}
@enduml
"""
)
*/

In [6]:

// print conenction between repos
enum class LinkType { parent, dependency }
data class Link(val src: String, val dist: String, val type: LinkType) {
    fun asD3() : String = "{ source: \"${src}\", target: \"${dist}\", type: \"${type}\" }"
}
data class Node(val id: String) {
    fun asD3() : String = "{ id: \"${id}\" }"
}

val edges = mutableSetOf<Link>()
for (repo in repos) {
    val parents = repo.artifacts
        .map { it.parent }
        .filterNotNull()

    repos
        .filter { it.artifacts.any { parents.contains(it) } }
        .filter { it.path != repo.path }
        .map { Link(repo.path, it.path, LinkType.parent) }
        .forEach { edges.add(it) }

    repo.dependencies
        .map { dep -> repos.find { it.artifacts.contains(dep) }?.path }
        .filterNotNull()
        .filter { it != repo.path }
        .map { Link(repo.path, it, LinkType.dependency) }
        .forEach { edges.add(it) }
}

val nodes = edges.map { listOf(it.src, it.dist) }.flatMap { it }.distinct().map { Node(it) }

println(nodes.joinToString(",\n\t", prefix = "export const nodes = [\n", postfix = "\n];") { it.asD3() })
println(edges.joinToString(",\n\t", prefix = "export const links = [\n", postfix = "\n];") { it.asD3() })


export const nodes = [
{ id: "openrewrite/rewrite-all" },
	{ id: "openrewrite/rewrite-csharp" },
	{ id: "openrewrite/rewrite" },
	{ id: "openrewrite/rewrite-python" },
	{ id: "openrewrite/rewrite-java-dependencies" },
	{ id: "openrewrite/rewrite-analysis" },
	{ id: "openrewrite/rewrite-apache" },
	{ id: "openrewrite/rewrite-templating" },
	{ id: "openrewrite/rewrite-static-analysis" },
	{ id: "openrewrite/rewrite-logging-frameworks" },
	{ id: "openrewrite/rewrite-build-gradle-plugin" },
	{ id: "openrewrite/rewrite-gradle-tooling-model" },
	{ id: "openrewrite/rewrite-cucumber-jvm" },
	{ id: "openrewrite/rewrite-docker" },
	{ id: "openrewrite/rewrite-dropwizard" },
	{ id: "openrewrite/rewrite-testing-frameworks" },
	{ id: "openrewrite/rewrite-feature-flags" },
	{ id: "openrewrite/rewrite-generative-ai" },
	{ id: "openrewrite/rewrite-github-actions" },
	{ id: "openrewrite/rewrite-gitlab" },
	{ id: "openrewrite/rewrite-gradle-plugin" },
	{ id: "openrewrite/rewrite-hibernate" },
	{ id: "ope